In [ ]:
import pandas as pd
test = pd.read_csv("/home/bruno/mestrado-ufrgs/mestrado-ufrgs-cmp617-tce/data/tce_licitations.csv").sample(100000)

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from yellowbrick.text import FreqDistVisualizer

def frequencia_palavras(lista_texto, top_n=15, orient='v'):
    # Substitui np.nan por uma string vazia
    lista_texto = np.where(pd.isnull(lista_texto), '', lista_texto)

    vectorizer = CountVectorizer()
    docs = vectorizer.fit_transform(lista_texto)
    features = vectorizer.get_feature_names_out()

    visualizer = FreqDistVisualizer(features=features, orient=orient, n=top_n)
    visualizer.fit(docs)
    visualizer.show()

frequencia_palavras(test["DS_OBJETO"].values, 30, 'h')
